In [ ]:
!pip install langgraph

In [34]:
from typing import TypedDict, List, Annotated
from langgraph.graph import END, StateGraph
import operator

In [35]:
class SimpleState(TypedDict):
    count: int
    sum: Annotated[int, operator.add]
    history: Annotated[List[int], operator.concat]


def increment(state: SimpleState) -> SimpleState:
    newCount = state["count"] + 1
    return {
        "count": newCount,
        "sum": newCount,
        "history": [newCount]
    }

In [14]:
def should_continue(state):
    if(state["count"] < 5):
        return "continue"
    else:
        return "stop"

In [ ]:
!pip install grandalf

In [36]:
graph = StateGraph(SimpleState)

graph.add_node("increment", increment)

graph.set_entry_point("increment")

graph.add_conditional_edges(
    "increment",
    should_continue,
    {
        "continue": "increment",
        "stop": END
    }
)


app = graph.compile()

print(app.get_graph().draw_mermaid())

state = {
    "count": 0,
    "sum" : 0,
    "history": []
}

result = app.invoke(state)
print(result)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	increment(increment)
	__end__([<p>__end__</p>]):::last
	__start__ --> increment;
	increment -. &nbsp;stop&nbsp; .-> __end__;
	increment -. &nbsp;continue&nbsp; .-> increment;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

{'count': 5, 'sum': 15, 'history': [1, 2, 3, 4, 5]}
